In [ ]:
!pip install transformers snorkel better_profanity textblob 

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from google.colab import drive
from nltk.corpus import stopwords
import nltk
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
import tensorflow_hub as hub
from transformers import BertTokenizer
from transformers import BertTokenizerFast, BertForSequenceClassification
from transformers import BertConfig
from transformers import BertModel, get_linear_schedule_with_warmup
from transformers import RobertaTokenizer
from transformers import RobertaConfig
from transformers import RobertaModel
from torch.utils.data import Dataset, DataLoader
import torch
from torch import nn
from sklearn.metrics import accuracy_score,f1_score
from sklearn.metrics import confusion_matrix
from collections import Counter
import spacy
from keras.preprocessing import sequence
from keras.preprocessing.text import Tokenizer
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
from keras.preprocessing import sequence
from tqdm import tqdm
from sklearn.metrics import mean_squared_error
from torch.optim import AdamW
from collections import defaultdict
from scipy.stats import entropy
from snorkel.labeling import labeling_function,PandasLFApplier
from snorkel.preprocess import preprocessor
from snorkel.preprocess.nlp import SpacyPreprocessor
from better_profanity import profanity
from textblob import TextBlob
from snorkel.labeling import LFAnalysis
from snorkel.analysis import get_label_buckets
from snorkel.labeling import LabelingFunction
from snorkel.labeling.model import MajorityLabelVoter
from snorkel.labeling.model import LabelModel
from snorkel.labeling import filter_unlabeled_dataframe
from snorkel.utils import probs_to_preds
from snorkel.preprocess.nlp import SpacyPreprocessor
import spacy
import random

In [ ]:
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

In [ ]:
drive.mount('/content/gdrive')

In [ ]:
using_colab = True

##Weak Supervison


In [ ]:
if using_colab:
  df = pd.read_csv('/content/gdrive/MyDrive/CS6471/reddit_comments.csv')
else:
  df = pd.read_csv('./reddit_comments.csv')


In [ ]:
#clean text
whitelist = set('abcdefghijklmnopqrstuvwxyz ABCDEFGHIJKLMNOPQRSTUVWXYZ1234567890')
def cleaner(x):
    x = x.replace("\n"," ")
    x = "".join(filter(whitelist.__contains__, x))
    x = x.lower()
    x = x.strip()
    return x

In [ ]:
df["clean_text"] = df["txt"].apply(cleaner)

In [ ]:
TOXIC = 1
NEUTRAL = -1
NONTOXIC = 0

In [ ]:
@labeling_function()
def contains_bad_words(x):
    return TOXIC if profanity.contains_profanity(x.txt) else NEUTRAL

In [ ]:
@preprocessor(memoize=True)
def comment_sentiment(x):
    scores = TextBlob(x.txt)
    x.polarity = scores.sentiment.polarity
    x.subjectivity = scores.sentiment.subjectivity
    return x

In [ ]:
#if polarity is >.9 more likely to be more non-toxic
@labeling_function(pre=[comment_sentiment])
def comment_polarity(x):
    return NONTOXIC if x.polarity > 0.9 else NEUTRAL

In [ ]:
#if subjectivity is >.7 more likely to be non-toxic
@labeling_function(pre=[comment_sentiment])
def comment_subjectivity(x):
    return NONTOXIC if x.subjectivity > 0.7 else NEUTRAL

In [ ]:
spacy = SpacyPreprocessor(text_field="txt", doc_field="doc", memoize=True)
#if comment contains URL more likely to be informative
@labeling_function(pre=[spacy])
def contains_url(x):
    """If comment contains url, label non-toxic, else abstain"""
    matcher = Matcher(nlp.vocab)
    pattern = [{"LIKE_URL": True}]
    matcher.add("p1", None, pattern)
    matches = matcher(x.doc)
    return NONTOXIC if len(matches)>0 else NEUTRAL

In [ ]:
#checks to see if toxic and nontoxic phrases are present.
#Got this from https://trishalaneeraj.github.io/2020-07-26/data-labeling-weak-supervision
def keyword_lookup(x, keywords, label):
    if any(word in x.clean_text.lower() for word in keywords):
        return label
    return NEUTRAL

def make_keyword_lf(keywords, label=TOXIC):
    return LabelingFunction(
        name=f"keyword_{keywords[0]}",
        f=keyword_lookup,
        resources=dict(keywords=keywords, label=label),
    )
    
with open('/content/gdrive/MyDrive/CS6471/badwords.txt') as f:
    toxic_stopwords = f.readlines()

toxic_stopwords = [x.strip() for x in toxic_stopwords] # len = 458
"""Comments mentioning at least one of Google's Toxic Stopwords 
https://code.google.com/archive/p/badwordslist/downloads are likely toxic"""
#Chceks to see if comments are toxic.
keyword_toxic_stopwords = make_keyword_lf(keywords=toxic_stopwords, label=TOXIC)

#Looks for keywords of please
keyword_pl = make_keyword_lf(keywords=["please", "plz", "pls", "pl"], label=NONTOXIC)
#Looks for keywords of thanks
keyword_thanks = make_keyword_lf(keywords=["thanks", "thank you", "thx", "tx"], label=NONTOXIC)

In [ ]:
lfs = [contains_bad_words, comment_polarity, comment_subjectivity, keyword_toxic_stopwords,keyword_pl,keyword_thanks]

applier = PandasLFApplier(lfs=lfs)
L_train = applier.apply(df)

In [ ]:
LFAnalysis(L=L_train, lfs=lfs).lf_summary()

In [ ]:
#Majority vote for prediction - probably poor model
majority_model = MajorityLabelVoter()
preds_train = majority_model.predict(L=L_train)
preds_train

In [ ]:
#More advanced model labeling model that will fit it based on true labels. How do we convert the output from snorkel to fit the true class labels?
label_model = LabelModel(cardinality=3, verbose=True)

true_labels = df["class"].to_numpy()
label_model.fit(L_train=L_train, n_epochs=1000, log_freq=200, seed=123)

In [ ]:
majority_acc = majority_model.score(L=L_train, Y=true_labels, tie_break_policy="abstain")[
    "accuracy"
]
print(f"{'Majority Vote Accuracy:':<25} {majority_acc * 100:.1f}%")

label_model_acc = label_model.score(L=L_train, Y=true_labels, tie_break_policy="abstain")[
    "accuracy"
]
print(f"{'Label Model Accuracy:':<25} {label_model_acc * 100:.1f}%")

In [ ]:
label_model_weights = np.around(label_model.get_weights(), 3)
probs_train = np.asarray(label_model.predict_proba(L_train))
preds = probs_to_preds(probs=probs_train)
filtered_df, probs_train_filtered = filter_unlabeled_dataframe(
    X=df, y=probs_train, L=L_train
)

In [ ]:
filtered_df = filtered_df[['post_id', 'txt','clean_text', 'class']]

In [ ]:
filtered_df.loc[filtered_df["class"] == -1, 'class'] = 'neutral'
filtered_df.loc[filtered_df["class"] == 0, 'class'] = 'nontoxic'
filtered_df.loc[filtered_df["class"] == 1, 'class'] = 'toxic'

In [ ]:
filtered_df.loc[filtered_df["class"] ==  'neutral', 'class'] = 1
filtered_df.loc[filtered_df["class"] == 'nontoxic', 'class'] = 0
filtered_df.loc[filtered_df["class"] == 'toxic', 'class'] = 2

In [ ]:
filename = 'reddit_weak_sup_dat.csv'
if using_colab:
  filtered_df.to_csv(f"/content/gdrive/MyDrive/CS6471/{filename}", index=False)
else:
  filtered_df.to_csv(f"/{filename}", index=False)

##Data Split for experiments 1,2,3

In [ ]:
if using_colab:
  df = pd.read_csv('/content/gdrive/MyDrive/CS6471/reddit_weak_sup_dat.csv')
else:
  df = pd.read_csv('./reddit_weak_sup_dat.csv')

In [ ]:
if using_colab: 
  actualdf = pd.read_csv('/content/gdrive/MyDrive/CS6471/reddit_comments.csv')
else:
  df = pd.read_csv('./reddit_comments.csv')

In [ ]:
def discretize(val,split):
  if(split[0][0]<=val<split[0][1]):
    return 0
  elif(split[1][0]<=val<split[1][1]):
    return -1
  else:
    return 1
split = [
    [min(df['offensiveness_score']),-0.33],
    [-0.33,0.33],
    [0.33,max(df['offensiveness_score'])]
    ]

In [ ]:
actualdf['class'] = actualdf['offensiveness_score'].apply(lambda x:discretize(x,split))

In [ ]:
actualdf.loc[actualdf['class'] == 1, 'class'] = 2
actualdf.loc[actualdf['class'] == -1, 'class'] =1
actualdf.loc[actualdf['class'] == 0, 'class'] = 0


In [ ]:
df = df.rename(columns={'class': 'predicted_toxic'})

In [ ]:
df = df[['comment_id','predicted_toxic' ]]

In [ ]:
dfe = pd.merge(df, actualdf, how='left', on=['comment_id','comment_id'], indicator=True)

###Experiment 1: Randomly fill nan values based on class distribution
###RUN ONLY FOR EXPERIMENT 1

In [ ]:
dfe['predicted_toxic'] = dfe['predicted_toxic'].fillna(5,inplace=True)
dfe["predicted_toxic"]=dfe["predicted_toxic"].apply(lambda x: random.randint(0,2) if x!=0 else x)

###Experiment 2: Drop unlabled data points
###RUN ONLY FOR EXPERIMENT 2

In [ ]:
dfe = dfe.dropna()

###Experiment 3: Fill unlabled with true labels
###RUN ONLY FOR EXPERIMENT 3

In [ ]:
dfe['predicted_toxic'].fillna(dfe['class'], inplace=True)

###Split Weak Supervised Data
###Continue to run here after picking experiment number

In [ ]:
#clean text
whitelist = set('abcdefghijklmnopqrstuvwxyz ABCDEFGHIJKLMNOPQRSTUVWXYZ1234567890')
def cleaner(x):
    x = x.replace("\n"," ")
    x = "".join(filter(whitelist.__contains__, x))
    x = x.lower()
    x = x.strip()
    return x

In [ ]:
dfe["clean_text"] = dfe["txt"].apply(cleaner)

In [ ]:
df_train, df_test = train_test_split(dfe, test_size=0.2, random_state=42)
y_train = list(df_train['predicted_toxic'])
y_test = list(df_test['class'])


# LSTM 

In [ ]:
vocab_size = 8000
max_len = 896

In [ ]:
#tokenization
tokenizer = Tokenizer(num_words = vocab_size)
tokenizer.fit_on_texts(df_train['clean_text'].values)#tokenization
tokenizer = Tokenizer(num_words = vocab_size)
tokenizer.fit_on_texts(df_train['clean_text'].values)

In [ ]:
train_encoding = sequence.pad_sequences(tokenizer.texts_to_sequences(df_train['clean_text'].values),maxlen = max_len)
test_encoding = sequence.pad_sequences(tokenizer.texts_to_sequences(df_test['clean_text'].values),maxlen = max_len)

In [ ]:
class comment_dataset(Dataset):
  def __init__(self,X,Y):
    self.X = X
    self.y = Y
  
  def __len__(self):
    return len(self.y)

  def __getitem__(self,idx):
    return torch.tensor(self.X[idx]),self.y[idx]

In [ ]:
batch_size = 32

train_dataset = comment_dataset(train_encoding,df_train['predicted_toxic'].values)
test_dataset = comment_dataset(test_encoding,df_test['class'].values)
train_dl = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_dl = DataLoader(test_dataset, batch_size=batch_size)

In [ ]:
class LSTM_fixed_len(torch.nn.Module) :
    def __init__(self, vocab_size, embedding_dim, hidden_dim) :
        super().__init__()
    
        self.embeddings = nn.Embedding(vocab_size, embedding_dim, padding_idx=0)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, batch_first=True)
        self.linear = nn.Linear(hidden_dim, 5)
        self.dropout = nn.Dropout(0.2)
        
    def forward(self, x):
        x = self.embeddings(x)
        x = self.dropout(x)
        lstm_out, (ht, ct) = self.lstm(x)
        return self.linear(ht[-1])

In [ ]:
model_fixed =  LSTM_fixed_len(vocab_size, 50, 50)

In [ ]:
def train_model(model, epochs=10, lr=0.001):
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    for i in range(epochs):
        model.train()
        sum_loss = 0.0
        total = 0
        for x, y in tqdm(train_dl):
            y_pred = model(x)
            loss = F.cross_entropy(y_pred, y)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            sum_loss += loss.item()*y.shape[0]
            total += y.shape[0]
        test_loss, test_acc, test_rmse = validation_metrics(model, test_dl)
        print("train loss %.3f, test loss %.3f, test accuracy %.3f, and test rmse %.3f" %(sum_loss/total, test_loss, test_acc, test_rmse))

def validation_metrics (model, test_dl):
    model.eval()
    correct = 0
    total = 0
    sum_loss = 0.0
    sum_rmse = 0.0
    for x, y in test_dl:
        x = x.long()
        y = y.long()
        y_hat = model(x)
        loss = F.cross_entropy(y_hat, y)
        pred = torch.max(y_hat, 1)[1]
        correct += (pred == y).float().sum()
        total += y.shape[0]
        sum_loss += loss.item()*y.shape[0]
        sum_rmse += np.sqrt(mean_squared_error(pred, y.unsqueeze(-1)))*y.shape[0]
    return sum_loss/total, correct/total, sum_rmse/total

In [ ]:
train_model(model_fixed, epochs=3, lr=0.01)

# Pretrained Models BERT, RoBERTA


In [ ]:
class RedditDataset(Dataset):

  def __init__(self, text, targets, tokenizer, max_len):
    self.text = text
    self.targets = targets
    self.tokenizer = tokenizer
    self.max_len = max_len
  
  def __len__(self):
    return len(self.text)
  
  def __getitem__(self, item):
    comment = self.text[item]
    target = self.targets[item]

    encoding_body = self.tokenizer.encode_plus(
      comment,
      add_special_tokens=True,
      max_length=self.max_len,
      return_token_type_ids=False,
      pad_to_max_length=True,
      return_attention_mask=True,
      return_tensors='pt',
      truncation = True
    )


    return {
      'comment_body': comment,
      'input_ids': encoding_body['input_ids'].flatten(),
      'attention_mask': encoding_body['attention_mask'].flatten(),
      'targets': torch.tensor(target, dtype=torch.long)
    }

def create_data_loader(df, tokenizer, max_len, batch_size, test_val):
  if test_val:
    test_class_name = 'class'
  else:
    test_class_name = 'predicted_toxic'
  ds = RedditDataset(
    text=df.clean_text.to_numpy(),
    targets=df[test_class_name].to_numpy(),
    tokenizer=tokenizer,
    max_len=max_len
  )

  return DataLoader(
    ds,
    batch_size=batch_size,
    num_workers=2
  )

In [ ]:
MAX_LEN = 512
BATCH_SIZE = 6
PRE_TRAINED_MODEL_NAME = 'bert-base-uncased'
tokenizer = BertTokenizer.from_pretrained(PRE_TRAINED_MODEL_NAME)
train_dataloader = create_data_loader(df_train, tokenizer = tokenizer, max_len = MAX_LEN, batch_size=BATCH_SIZE, test_val=False)
test_dataloader = create_data_loader(df_test, tokenizer = tokenizer, max_len = MAX_LEN, batch_size=BATCH_SIZE, test_val=True)

In [ ]:
nb_labels = 3
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

class RedditClassifier(nn.Module):
  def __init__(self,nb_labels):
    super(RedditClassifier,self).__init__()
    self.bert_body = BertModel.from_pretrained(PRE_TRAINED_MODEL_NAME)
    #self.bert_body = RobertaModel.from_pretrained(PRE_TRAINED_MODEL_NAME)


    self.drop = nn.Dropout(p=0.3)
    self.out = nn.Linear(self.bert_body.config.hidden_size, nb_labels)


  def forward(self, input_ids, attention_mask):
    _, pooled_output = self.bert_body(
        input_ids = input_ids,
        attention_mask = attention_mask,
        return_dict = False
    )

    output = self.drop(pooled_output)
    return self.out(output)

In [ ]:
EPOCHS = 1
model = RedditClassifier(nb_labels)
model.to(device)
optimizer = AdamW(model.parameters(), lr=2e-5)
total_steps = len(train_dataloader) * EPOCHS

scheduler = get_linear_schedule_with_warmup(
  optimizer,
  num_warmup_steps=0,
  num_training_steps=total_steps
)

loss_fn = nn.CrossEntropyLoss().to(device)

In [ ]:
def train_epoch(
  model, 
  data_loader, 
  loss_fn, 
  optimizer, 
  device, 
  scheduler, 
  n_examples
):
  model = model.train()

  losses = []
  correct_predictions = 0
  
  for d in tqdm(data_loader,position = 0,leave = True):
    input_ids = d["input_ids"].to(device)
    attention_mask = d["attention_mask"].to(device)
    targets = d["targets"].to(device)


    outputs = model(
      input_ids=input_ids,
      attention_mask=attention_mask
    )
    _, preds = torch.max(outputs, dim=1)
    loss = loss_fn(outputs, targets)

    correct_predictions += torch.sum(preds == targets)
    losses.append(loss.item())

    loss.backward()
    nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
    optimizer.step()
    scheduler.step()
    optimizer.zero_grad()

  return correct_predictions.double() / n_examples, np.mean(losses)

In [ ]:
def eval_model(model, data_loader, loss_fn, device, n_examples):
  model = model.eval()

  losses = []
  correct_predictions = 0

  with torch.no_grad():
    for d in data_loader:
      input_ids = d["input_ids"].to(device)
      attention_mask = d["attention_mask"].to(device)
      targets = d["targets"].to(device)



      outputs = model(
        input_ids=input_ids,
        attention_mask=attention_mask,

      )
      x = outputs
      _, preds = torch.max(outputs, dim=1)

      loss = loss_fn(outputs, targets)

      correct_predictions += torch.sum(preds == targets)
      losses.append(loss.item())

  return correct_predictions.double() / n_examples, np.mean(losses)

In [ ]:
history = defaultdict(list)
best_accuracy = 0

for epoch in range(EPOCHS):

  print(f'Epoch {epoch + 1}/{EPOCHS}')
  print('-' * 10)

  train_acc, train_loss = train_epoch(
    model,
    train_dataloader,    
    loss_fn, 
    optimizer, 
    device, 
    scheduler, 
    len(df_train)
  )

  print(f'Train loss {train_loss} accuracy {train_acc}')

  test_acc, test_loss = eval_model(
    model,
    test_dataloader,
    loss_fn, 
    device, 
    len(df_test)
  )

  print(f'Test loss {test_loss} accuracy {test_acc}')
  print()

  history['train_acc'].append(train_acc)
  history['train_loss'].append(train_loss)
  history['test_acc'].append(test_acc)
  history['test_loss'].append(test_loss)

  if test_acc > best_accuracy:
    torch.save(model.state_dict(), 'best_model_state.bin')
    best_accuracy = test_acc